## 1. セットアップ

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import torch

# プロジェクトルートをパスに追加
sys.path.append('/workspace/atma_22_ca/')

from configs.config import *
from src.runner import Runner
from src.model_arcface import ModelArcFace
from src.util import Logger, Validation, Q1Q2Validator, Metric

print(f"Input dir: {DIR_INPUT}")
print(f"Model dir: {DIR_MODEL}")
print(f"Submission dir: {DIR_SUBMISSIONS}")

## 2. データ読み込み

In [ ]:
df_train = pd.read_csv(FILE_TRAIN_META)
df_test = pd.read_csv(FILE_TEST_META)

# グループ化
# df_train['group'] = df_train['quarter'] + '_' + df_train['session'].astype(str)

# インデックス付与（前処理済み画像読み込み用）
df_train = df_train.reset_index(drop=False) 

## 3. Logger初期化

In [ ]:
logger = Logger('logs/')
logger.info("Logger初期化完了")

## 4. モデルパラメータ設定

In [ ]:
# 基本パラメータ（閾値は後で調整）
params_base = {
    'model_name': 'efficientnet_b0',
    'embedding_dim': 512,
    'img_size': 224,
    'batch_size': 64,
    'epochs': 20,
    'lr': 1e-3,
    'weight_decay': 1e-4,
    'arcface_s': 30.0,
    'arcface_m': 0.5,
    'threshold': 0.5,  # 初期値、後で最適化
    'use_ema': True,
    'ema_decay': 0.995,
    'num_workers': 8,
}

print("パラメータ設定完了")
print(f"初期閾値: {params_base['threshold']}")

## 5. Step 1: Unknown CVで閾値探索

Q1/Q2分離CVでunknown判定能力を評価し、最適な閾値を見つけます。

In [ ]:
logger.info("="*80)
logger.info("Step 1: Unknown CVで閾値探索")
logger.info("="*80)

# 複数の閾値を試す
threshold_candidates = [0.3, 0.4, 0.5, 0.6, 0.7]
threshold_results = []

for threshold in threshold_candidates:
    logger.info(f"\n{'='*80}")
    logger.info(f"閾値 {threshold} で評価中...")
    logger.info(f"{'='*80}")
    
    # パラメータ設定
    params = params_base.copy()
    params['threshold'] = threshold
    
    # Unknown CV用のvalidator
    validator_unknown = Q1Q2Validator(quarter_col='quarter')
    
    # Runner作成
    run_name = f'unknown_threshold_{int(threshold*10)}'
    runner_unknown = Runner(
        run_name=run_name,
        model_cls=ModelArcFace,
        params=params,
        df_train=df_train,
        df_test=df_test,
        cv_setting={'validator': validator_unknown},
        logger=logger
    )
    
    # 学習
    runner_unknown.train_cv()
    
    # OOF予測を取得
    oof_pred_path = f'models/{run_name}/va_pred.pkl'
    oof_df = pd.read_pickle(oof_pred_path)
    
    # 元のデータとマージしてquarter情報を取得
    oof_df = oof_df.merge(
        df_train[['quarter']],
        left_index=True,
        right_index=True,
        how='left'
    )
    
    # Fold 0: Q1検証（選手0がunknown）
    fold0_mask = oof_df['quarter'].astype(str).str.startswith('Q1')
    metrics_fold0 = Metric.unknown_metrics(
        oof_df[fold0_mask]['label_id'].values,
        oof_df[fold0_mask]['pred'].values,
        unknown_player_id=0
    )
    
    # Fold 1: Q2検証（選手5がunknown）
    fold1_mask = oof_df['quarter'].astype(str).str.startswith('Q2')
    metrics_fold1 = Metric.unknown_metrics(
        oof_df[fold1_mask]['label_id'].values,
        oof_df[fold1_mask]['pred'].values,
        unknown_player_id=5
    )
    
    # 平均スコア
    avg_unknown_f1 = (metrics_fold0['unknown_f1'] + metrics_fold1['unknown_f1']) / 2
    avg_known_f1 = (metrics_fold0['known_macro_f1'] + metrics_fold1['known_macro_f1']) / 2
    
    # 結果を保存
    threshold_results.append({
        'threshold': threshold,
        'unknown_f1': avg_unknown_f1,
        'known_f1': avg_known_f1,
        'fold0_unknown_f1': metrics_fold0['unknown_f1'],
        'fold0_known_f1': metrics_fold0['known_macro_f1'],
        'fold1_unknown_f1': metrics_fold1['unknown_f1'],
        'fold1_known_f1': metrics_fold1['known_macro_f1']
    })
    
    logger.info(f"\n閾値 {threshold} の結果:")
    logger.info(f"  Unknown F1 (平均): {avg_unknown_f1:.4f}")
    logger.info(f"  既知選手 F1 (平均): {avg_known_f1:.4f}")

print("\n閾値探索完了")

## 6. 閾値探索結果の確認

In [ ]:
# 結果をDataFrameに変換
df_threshold_results = pd.DataFrame(threshold_results)

print("\n閾値探索結果:")
print(df_threshold_results[['threshold', 'unknown_f1', 'known_f1']])

# 最適閾値を選択（Unknown F1が最大）
best_idx = df_threshold_results['unknown_f1'].idxmax()
optimal_threshold = df_threshold_results.loc[best_idx, 'threshold']
best_unknown_f1 = df_threshold_results.loc[best_idx, 'unknown_f1']

logger.info("\n" + "="*80)
logger.info(f"最適閾値: {optimal_threshold}")
logger.info(f"  Unknown F1: {best_unknown_f1:.4f}")
logger.info("="*80)

print(f"\n✅ 最適閾値: {optimal_threshold}")

## 7. Step 2: 最適閾値で通常CVを学習

全選手（0-10）を学習する通常の3-fold CVを実行します。

In [ ]:
logger.info("\n" + "="*80)
logger.info("Step 2: 最適閾値で通常3-fold CVを学習")
logger.info("="*80)

# 最適閾値を適用
params_optimized = params_base.copy()
params_optimized['threshold'] = optimal_threshold

# 通常CV用のvalidator
validator_normal = Validation.create_validator(
    method='stratified_group',
    n_splits=3,
    shuffle=True,
    random_state=42
)

# Runner作成
timestamp = datetime.now().strftime('%Y%m%d%H%M')
run_name_final = f'final_optimized_{timestamp}'

runner_final = Runner(
    run_name=run_name_final,
    model_cls=ModelArcFace,
    params=params_optimized,
    df_train=df_train,
    df_test=df_test,
    cv_setting={'validator': validator_normal, 'group_col': 'quarter'},
    logger=logger
)

logger.info(f"最適閾値 {optimal_threshold} で学習開始")

# 学習
runner_final.train_cv()

print("\n通常CV学習完了")

## 8. OOF評価

In [ ]:
# OOF評価
scores, oof_score = runner_final.metric_cv()

logger.info("\n" + "="*80)
logger.info("最終モデル評価結果")
logger.info("="*80)
logger.info(f"OOFスコア (Macro F1): {oof_score:.5f}")
logger.info(f"各Foldスコア: {[f'{s:.5f}' for s in scores]}")
logger.info(f"使用閾値: {optimal_threshold}")
logger.info("="*80)

print(f"\n✅ OOFスコア: {oof_score:.5f}")

## 9. Step 3: テストデータで予測

通常CVの3モデルのみで予測します（全選手を学習済みなので安全）。

In [ ]:
logger.info("\n" + "="*80)
logger.info("Step 3: テストデータで予測")
logger.info("="*80)
logger.info("⚠️ 通常CVの3モデルのみ使用（全選手0-10を学習済み）")

# 予測
submission = runner_final.predict_cv()

print("\n予測完了")
print(f"予測サンプル数: {len(submission):,}")

## 10. 予測結果の確認

In [ ]:
# 予測分布
print("\n予測分布:")
pred_dist = submission['label_id'].value_counts().sort_index()

for label, count in pred_dist.items():
    label_name = f"選手{label}" if label != -1 else "Unknown"
    percentage = count / len(submission) * 100
    print(f"  {label_name}: {count:,}サンプル ({percentage:.1f}%)")

# Unknown判定の割合
unknown_count = (submission['label_id'] == -1).sum()
unknown_ratio = unknown_count / len(submission) * 100
print(f"\nUnknown判定率: {unknown_ratio:.2f}%")

## 11. 提出ファイル保存

In [ ]:
# 提出ファイル保存
submission_path = runner_final.save_submission(submission)

logger.info("\n" + "="*80)
logger.info("全工程完了！")
logger.info("="*80)
logger.info(f"提出ファイル: {submission_path}")
logger.info(f"最終OOFスコア: {oof_score:.5f}")
logger.info(f"最適閾値: {optimal_threshold}")
logger.info(f"使用モデル数: 3 (通常CVのみ)")
logger.info(f"Unknown判定率: {unknown_ratio:.2f}%")
logger.info("="*80)

print(f"\n✅ 提出ファイル: {submission_path}")
print(f"✅ OOFスコア: {oof_score:.5f}")
print(f"✅ 最適閾値: {optimal_threshold}")

## まとめ

### 実行した内容
1. **Unknown CVで閾値探索**: 複数の閾値（0.3, 0.4, 0.5, 0.6, 0.7）を試してUnknown判定能力を評価
2. **最適閾値で通常CV**: Unknown F1が最大の閾値を使って全選手（0-10）で3-fold CV学習
3. **予測**: 通常CVの3モデルのみで予測（全選手を学習済みなので安全）

### 重要ポイント
- ✅ Unknown CVは評価と閾値チューニング専用
- ✅ 最終予測は通常CVのモデルのみ（選手0と5も正しく予測可能）
- ✅ Unknown判定能力と既知選手の識別能力を両立